In [96]:
import os
import random
from dataclasses import dataclass

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [97]:
@dataclass
class TrainCfg:
    # You can swap this to a larger model if your GPU allows:
    # e.g. 'microsoft/deberta-v3-large'
    # If you want to move closer to the 9th-place approach,
    # you could later switch to a Gemma model:
    # 'google/gemma-2-2b-it' (or 9b on Kaggle with 4-bit).
    model_name: str = "roberta-base"

    max_length: int = 512
    train_batch_size: int = 4
    valid_batch_size: int = 4
    learning_rate: float = 2e-5
    epochs: int = 3
    seed: int = 42

    # Paths (Kaggle-compatible structure)
    train_path: str = "data/train.csv"
    test_path: str = "data/test.csv"
    sub_path: str = "data/ours_submission.csv"

    device: torch.device = torch.device(
        "cuda" if torch.cuda.is_available() else "cpu"
    )

cfg = TrainCfg()
set_seed(cfg.seed)
print("Using device:", cfg.device)


Using device: cuda


In [98]:
from transformers import DataCollatorWithPadding
import ast

def process_text(text):
    """
    Robustly convert the CSV field to plain text.
    Handles:
      - None / NaN
      - actual Python lists
      - stringified lists like '["hello", "world"]'
      - malformed '[...]' that can't be parsed (falls back gracefully)
    """
    # 1. Missing values
    if text is None:
        return ""
    if isinstance(text, float) and np.isnan(text):
        return ""

    # 2. Already a list -> join its elements
    if isinstance(text, list):
        return " ".join("" if t is None else str(t) for t in text)

    # 3. Convert anything else to str
    s = str(text)

    # 4. Try to parse stringified list safely
    if s.startswith("[") and s.endswith("]"):
        try:
            parsed = ast.literal_eval(s)
        except Exception:
            # Not a valid Python literal, just strip the brackets and treat as text
            return s.strip("[]")

        # If it parsed successfully, handle lists or other literals
        if isinstance(parsed, list):
            return " ".join("" if t is None else str(t) for t in parsed)
        else:
            # e.g. parsed into a single string or number
            return str(parsed)

    # 5. Plain string
    return s



class LMSYSDataset(Dataset):
    def __init__(self, df, tokenizer, max_length, is_test=False):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test
        
        # Convert to records for speed
        self.data = self.df[["prompt", "response_a", "response_b"]].to_dict("records")

        # Pre-compute labels
        if not self.is_test:
            self.labels = []
            for _, row in self.df.iterrows():
                if row["winner_model_a"] == 1:
                    label = 0
                elif row["winner_model_b"] == 1:
                    label = 1
                else:
                    label = 2
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data[idx]
        
        # Safe access to separator token (fallback to space if None)
        sep = self.tokenizer.sep_token if self.tokenizer.sep_token is not None else " "
        
        # FORCE conversion to standard Python string to avoid TypeErrors
        # This fixes the crash on rows with weird types (e.g. numpy str_, nan, etc.)
        p = str(row.get("prompt", ""))
        ra = str(row.get("response_a", ""))
        rb = str(row.get("response_b", ""))
        
        text = f"{p} {sep} {ra} {sep} {rb}"

        # Tokenize
        try:
            inputs = self.tokenizer(
                text,
                truncation=True,
                max_length=self.max_length,
                add_special_tokens=True,
            )
        except TypeError:
            # Fallback for extremely weird edge cases: treat as empty string
            print(f"Warning: Tokenization failed at idx {idx}. Using empty text.")
            print(f"Offending text: prompt={p}, response_a={ra}, response_b={rb}")
            inputs = self.tokenizer(
                "",
                truncation=True,
                max_length=self.max_length,
                add_special_tokens=True,
            )

        item = {
            "input_ids": torch.tensor(inputs["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(inputs["attention_mask"], dtype=torch.long),
        }

        if not self.is_test:
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)

        return item


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = F.softmax(torch.tensor(logits), dim=-1).numpy()

    num_classes = probs.shape[1]
    labels = np.array(labels, dtype=int)
    labels_oh = np.eye(num_classes)[labels]

    return {"log_loss": log_loss(labels_oh, probs)}


In [99]:
import ast
import numpy as np
import pandas as pd
import re

def strip_surrogates(text: str) -> str:
    """Remove characters that cannot be encoded in UTF-8 (surrogates etc)."""
    if not isinstance(text, str):
        text = str(text)
    return text.encode("utf-8", "replace").decode("utf-8")

def strip_control_chars(s: str) -> str:
    """Remove ASCII control chars except \n and \t."""
    return re.sub(r"[\x00-\x08\x0B-\x1F\x7F]", "", s)

def safe_literal_list(text: str):
    """Safely parse stringified list like '["a", "b"]', else return None."""
    if text.startswith("[") and text.endswith("]"):
        try:
            parsed = ast.literal_eval(text)
            if isinstance(parsed, list):
                return parsed
        except Exception:
            return None
    return None

def clean_text(x):
    """Complete cleaning pipeline."""
    # None / NaN
    if x is None:
        return ""
    if isinstance(x, float) and np.isnan(x):
        return ""

    # Already a Python list
    if isinstance(x, list):
        return " ".join(strip_surrogates(strip_control_chars(str(t))) for t in x)

    # Convert to string
    s = str(x).strip()

    # Try parsing JSON-like list
    parsed_list = safe_literal_list(s)
    if parsed_list is not None:
        # Join list elements
        return " ".join(strip_surrogates(strip_control_chars(str(t))) for t in parsed_list)

    # Strip unicode surrogates and control characters
    s = strip_surrogates(s)
    s = strip_control_chars(s)

    return s


print("Loading data...")

train_df = pd.read_csv(cfg.train_path)
test_df = pd.read_csv(cfg.test_path)

print("Original train shape:", train_df.shape)
print("Original test shape:", test_df.shape)

for df_name, df in [("train", train_df), ("test", test_df)]:
    # 1) Drop NaN rows
    df.dropna(subset=["prompt", "response_a", "response_b"], inplace=True)

    # 2) Clean text fully
    for col in ["prompt", "response_a", "response_b"]:
        df[col] = df[col].apply(clean_text)

    # 3) Drop empty/whitespace rows
    mask_empty = (
        df["prompt"].str.strip().eq("") |
        df["response_a"].str.strip().eq("") |
        df["response_b"].str.strip().eq("")
    )
    to_drop = mask_empty.sum()
    if to_drop > 0:
        print(f"{df_name}: Dropping {to_drop} empty rows")
        df.drop(df[mask_empty].index, inplace=True)

    df.reset_index(drop=True, inplace=True)

    print(f"After cleaning, {df_name} shape:", df.shape)


Loading data...
Original train shape: (57477, 9)
Original test shape: (3, 4)
train: Dropping 26 empty rows
After cleaning, train shape: (57451, 9)
After cleaning, test shape: (3, 4)


In [100]:
print("Loading tokenizer & model...")
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

# Ensure pad_token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Split data
train_split, val_split = train_test_split(
    train_df,
    test_size=0.1,
    random_state=cfg.seed,
    shuffle=True,
)

# Initialize Datasets (Now fast because we don't tokenize yet)
train_dataset = LMSYSDataset(train_split, tokenizer, cfg.max_length, is_test=False)
valid_dataset = LMSYSDataset(val_split, tokenizer, cfg.max_length, is_test=False)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(valid_dataset)}")

model = AutoModelForSequenceClassification.from_pretrained(
    cfg.model_name,
    num_labels=3,
)

if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id

# Use DataCollator to pad batches dynamically to the longest sequence in the BATCH
# (instead of padding everything to max_length=1024, which wastes GPU memory)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

args = TrainingArguments(
    output_dir=f"./outputs/{cfg.model_name.replace('/', '_')}_finetuned",
    num_train_epochs=cfg.epochs,
    per_device_train_batch_size=cfg.train_batch_size,
    per_device_eval_batch_size=cfg.valid_batch_size,
    learning_rate=cfg.learning_rate,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="log_loss",
    greater_is_better=False,
    logging_steps=100,
    save_total_limit=1,
    report_to="none",
    fp16=True, # Recommended for T4/P100 GPUs (Kaggle) to save memory/speed up
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator, # <--- Added this
)

print("Starting training...")
trainer.train()
print("Training done.")

Loading tokenizer & model...
Train samples: 51705, Val samples: 5746


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\User\AppData\Local\Temp\ipykernel_28416\628969652.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


Epoch,Training Loss,Validation Loss,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,1.095900,1.097005,1.097114,37.673000,152.523000,38.144000
2,1.094900,1.096594,1.096550,55.778400,103.015000,25.763000
3,1.099400,1.097170,1.097041,54.686000,105.073000,26.277000


Training done.


In [106]:
cfg = TrainCfg(
    model_name="distilroberta-base", 
    epochs=20,
    learning_rate=3e-5, 
    train_batch_size=64, 
    valid_batch_size=64
)

In [107]:
print("Loading tokenizer & model...")
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

# Ensure pad_token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Split data
train_split, val_split = train_test_split(
    train_df,
    test_size=0.1,
    random_state=cfg.seed,
    shuffle=True,
)

# Initialize Datasets (Now fast because we don't tokenize yet)
train_dataset = LMSYSDataset(train_split, tokenizer, cfg.max_length, is_test=False)
valid_dataset = LMSYSDataset(val_split, tokenizer, cfg.max_length, is_test=False)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(valid_dataset)}")

model = AutoModelForSequenceClassification.from_pretrained(
    cfg.model_name,
    num_labels=3,
)

if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id

# Use DataCollator to pad batches dynamically to the longest sequence in the BATCH
# (instead of padding everything to max_length=1024, which wastes GPU memory)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

args = TrainingArguments(
    output_dir=f"./outputs/{cfg.model_name.replace('/', '_')}_finetuned",
    num_train_epochs=cfg.epochs,
    per_device_train_batch_size=cfg.train_batch_size,
    per_device_eval_batch_size=cfg.valid_batch_size,
    learning_rate=cfg.learning_rate,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="log_loss",
    greater_is_better=False,
    logging_steps=100,
    save_total_limit=1,
    report_to="none",
    fp16=True, # Recommended for T4/P100 GPUs (Kaggle) to save memory/speed up
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator, # <--- Added this
)

print("Starting training...")
trainer.train()
print("Training done.")

Loading tokenizer & model...
Train samples: 51705, Val samples: 5746


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\User\AppData\Local\Temp\ipykernel_28416\3441798207.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


Epoch,Training Loss,Validation Loss,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,1.084200,1.077140,1.077139,20.694200,277.663000,4.349000
2,1.075300,1.092544,1.092539,20.744000,276.996000,4.339000
3,1.044400,1.089451,1.089446,20.872800,275.286000,4.312000
4,0.991700,1.113672,1.113669,20.838100,275.745000,4.319000
5,0.897500,1.236092,1.236091,20.459400,280.848000,4.399000
6,0.797000,1.368358,1.368353,20.805100,276.183000,4.326000
7,0.689400,1.448684,1.448684,20.817200,276.022000,4.323000
8,0.617900,1.850870,1.850864,21.085200,272.514000,4.268000
9,0.525200,2.009864,2.009868,20.692700,277.682000,4.349000
10,0.433600,2.180743,2.180742,20.479900,280.568000,4.395000


Training done.


In [ ]:
cfg = TrainCfg(
    model_name="distilroberta-base", 
    
    learning_rate=3e-5, 
    train_batch_size=16, 
    valid_batch_size=16
)
trainer.train(resume_from_checkpoint=True)

In [101]:
print("Starting inference with A/B swap TTA...")

# 1. Normal test dataset: (prompt, A, B)
test_ds_normal = LMSYSDataset(test_df, tokenizer, cfg.max_length, is_test=True)

# 2. Swapped test dataset: (prompt, B, A)
test_df_swapped = test_df.copy()
test_df_swapped[["response_a", "response_b"]] = test_df_swapped[["response_b", "response_a"]]
test_ds_swapped = LMSYSDataset(test_df_swapped, tokenizer, cfg.max_length, is_test=True)

# Predict
preds_normal = trainer.predict(test_ds_normal).predictions
preds_swapped = trainer.predict(test_ds_swapped).predictions

probs_normal = F.softmax(torch.tensor(preds_normal), dim=-1).numpy()
probs_swapped = F.softmax(torch.tensor(preds_swapped), dim=-1).numpy()

# For swapped, model's "class 0" means "first response wins" which is B in original,
# and "class 1" means "second response wins" which is A in original.
# So we need to swap back the first two probability columns.
probs_swapped_fixed = np.zeros_like(probs_swapped)
probs_swapped_fixed[:, 0] = probs_swapped[:, 1]  # B->A
probs_swapped_fixed[:, 1] = probs_swapped[:, 0]  # A->B
probs_swapped_fixed[:, 2] = probs_swapped[:, 2]  # tie unchanged

# Ensemble
final_probs = (probs_normal + probs_swapped_fixed) / 2.0

submission = pd.DataFrame(
    {
        "id": test_df["id"].values,
        "winner_model_a": final_probs[:, 0],
        "winner_model_b": final_probs[:, 1],
        "winner_tie": final_probs[:, 2],
    }
)

os.makedirs(os.path.dirname(cfg.sub_path), exist_ok=True)
submission.to_csv(cfg.sub_path, index=False)
print("Saved submission to:", cfg.sub_path)
submission.head()


Starting inference with A/B swap TTA...


Saved submission to: data/ours_submission.csv


,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.347363,0.347363,0.305274
1,211333,0.347366,0.347366,0.305269
2,1233961,0.347362,0.347362,0.305276
